In [16]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pandas as pd

import pickle
import jieba
import h5py

SOS_token = 0
EOS_token = 1

### load all data

In [17]:
import dataProcess as dp
h5py_file = h5py.File('../data/train_afterProcess.h5py','r')
pairs = h5py_file['pairs']

print(pairs[0][0].decode('utf-8'))
print(pairs[0][1].decode('gb2312'))

inputlang = dp.Lang('en')
outputlang = dp.Lang('zh')
inputlang.load('../data/en_train.pkl')
outputlang.load('../data/zh_train.pkl')

####测试用，上面两行注释的语句在真正运行的时候要用到的##################################################################
# pairs = pairs[0:1000]
# for pair in pairs:
#     inputlang.addSentence(pair[0].decode('utf-8'))
#     outputlang.addSentence(pair[1].decode('gb2312'))
    
print(inputlang.name,inputlang.n_words)
print(outputlang.name,outputlang.n_words)
#h5py_file.close()


a pair of red crowned cranes have staked out their nesting territory
 一 对 丹 顶 鹤 正 监 视 着 它 们 的 筑 巢 领 地 
error: Name error------------------------------!
error: Name error------------------------------!
en 388091
zh 6754


In [18]:
#cut some unfrequency words
print(len(inputlang.word2count))
print(len(inputlang.word2index))
print(len(inputlang.index2word))
all_en_words = inputlang.word2count.copy()
for word in all_en_words:
    if  all_en_words[word] <= 10:
        inputlang.word2count.pop(word)
        index = inputlang.word2index[word]
        inputlang.word2index.pop(word)
        inputlang.index2word.pop(index)
        
print(len(inputlang.word2count))
print(len(inputlang.word2index))
print(len(inputlang.index2word))

388089
388089
388091
82101
82101
82103


### load test data

In [ ]:
import dataProcess as dp
h5py_file = h5py.File('../data/train_afterProcess.h5py','r')
pairs = h5py_file['pairs']

print(pairs[0][0].decode('utf-8'))
print(pairs[0][1].decode('gb2312'))

inputlang = dp.Lang('en')
outputlang = dp.Lang('zh')
# inputlang.load('../data/en_train.pkl')
# outputlang.load('../data/zh_train.pkl')

####测试用，上面两行注释的语句在真正运行的时候要用到的##################################################################
pairs = pairs[0:1000]
for pair in pairs:
    inputlang.addSentence(pair[0].decode('utf-8'))
    outputlang.addSentence(pair[1].decode('gb2312'))
    
print(inputlang.name,inputlang.n_words)
print(outputlang.name,outputlang.n_words)
#h5py_file.close()


388089
388089
388091
82101
82101
82103


# training and evaluate

In [5]:

import models as m

hidden_size = 256
encoder1 = m.EncoderRNN(inputlang.n_words, hidden_size)
attn_decoder1 = m.AttnDecoderRNN(hidden_size, outputlang.n_words,
                               1, dropout_p=0.1)

if m.use_cuda:
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

m.trainIters(encoder1, attn_decoder1, inputlang, outputlang,pairs, n_iters = 8000000, \
             plot_every = 100, print_every=1000, save_model_every=400000)

1m 38s (- 130m 13s) (1000 1%) 3.7346
3m 0s (- 117m 24s) (2000 2%) 3.3245
4m 18s (- 110m 39s) (3000 3%) 2.8743
5m 38s (- 107m 2s) (4000 5%) 2.5834
7m 1s (- 105m 28s) (5000 6%) 2.3345
8m 28s (- 104m 35s) (6000 7%) 2.1141
10m 11s (- 106m 21s) (7000 8%) 1.8685
11m 53s (- 106m 58s) (8000 10%) 1.7289
13m 35s (- 107m 16s) (9000 11%) 1.4711
15m 14s (- 106m 40s) (10000 12%) 1.3665
16m 35s (- 104m 2s) (11000 13%) 1.2610
17m 55s (- 101m 35s) (12000 15%) 1.0647
19m 4s (- 98m 16s) (13000 16%) 0.9653
20m 15s (- 95m 28s) (14000 17%) 0.9568
21m 33s (- 93m 24s) (15000 18%) 0.8217
22m 52s (- 91m 28s) (16000 20%) 0.8044
24m 8s (- 89m 29s) (17000 21%) 0.5796
25m 49s (- 88m 56s) (18000 22%) 0.5767
27m 34s (- 88m 31s) (19000 23%) 0.5180
29m 6s (- 87m 18s) (20000 25%) 0.4798
30m 26s (- 85m 31s) (21000 26%) 0.3960
31m 48s (- 83m 51s) (22000 27%) 0.3309
33m 18s (- 82m 32s) (23000 28%) 0.2961
34m 56s (- 81m 30s) (24000 30%) 0.2452
36m 40s (- 80m 41s) (25000 31%) 0.2579
38m 23s (- 79m 44s) (26000 32%) 0.2516
40m

### train test data

In [ ]:
import models as m

hidden_size = 256
encoder1 = m.EncoderRNN(inputlang.n_words, hidden_size)
attn_decoder1 = m.AttnDecoderRNN(hidden_size, outputlang.n_words,
                               1, dropout_p=0.1)

if m.use_cuda:
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

m.trainIters(encoder1, attn_decoder1, inputlang, outputlang,pairs, n_iters = 80000, \
             plot_every=100, print_every=1000, save_model_every=40000)

In [6]:
m.evaluateRandomly(encoder1,attn_decoder1,inputlang,outputlang,pairs,n = 10)

> a couple of what ?
=  一 对 什 么 ？ 
<  一 对 什 么 ？  <EOS>

> a little garlic some paprika
=  一 小 块 大 蒜 一 些 辣 椒 
<  一 小 块 大 蒜 一 些 辣 椒  <EOS>

> i took one in your driveway half an hour ago .
=  半 个 小 时 前 我 开 车 过 来 的 还 吃 了 一 颗 。 
<  半 个 小 时 前 我 开 车 过 来 的 还 吃 了 一 颗 。  <EOS>

> painter s due in an hour .
=  一 小 时 内 油 漆 匠 就 能 做 完 。 
<  一 小 时 内 油 漆 匠 就 能 做 完 。  <EOS>

> one second . wait . i ii be right back !
=  一 小 会 儿 。 稍 等 。 我 马 上 回 来 ！ 
<  一 小 会 儿 。 稍 等 。 我 马 上 回 来 ！  <EOS>

> teenagers fornicating in a liverpool basement .
=  一 对 在 利 物 浦 地 下 室 私 通 的 青 少 年 。 
<  一 对 在 利 物 浦 地 下 室 私 通 的 青 少 年 。  <EOS>

> couple of people ? no a couple of antelope .
=  一 对 夫 妇 ？ 不 ， 是 一 对 羚 羊 。 
<  一 对 夫 妇 ？ 不 ， 是 一 对 羚 羊 。  <EOS>

> couple of pretty ladies both trained assassins .
=  一 对 漂 亮 女 性 ， 都 受 训 成 为 刺 客 。 
<  一 对 漂 亮 女 性 ， 都 受 训 成 为 刺 客 。  <EOS>

> a loving young couple cleaved together in terror 
=  一 对 年 轻 爱 人 紧 抱 在 一 起 陷 入 恐 惧 ， 
<  一 对 年 轻 爱 人 紧 抱 在 一 起 陷 入 恐 惧 ，  <EOS>

> one that you wrote to 

# load model and evaluate

In [15]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import models as m
#filter some Pair for test
hidden_size = 256
# encoder2 = m.EncoderRNN(inputlang.n_words, hidden_size)
# attn_decoder2 = m.AttnDecoderRNN(hidden_size, outputlang.n_words,
#                                1, dropout_p=0.1)
# encoder2.load_state_dict(torch.load('../models/encoder.model8000'))
# attn_decoder2.load_state_dict(torch.load('../models/decoder.model8000'))

# encoder2 = torch.load('../models/encoder.model800')
# decoder2 = torch.load('../models/decoder.model800')
encoder2 = torch.load('../models/encoder{0}.m{1}'.format('first',60000))
decoder2 = torch.load('../models/decoder{0}.m{1}'.format('first',60000))

In [19]:
m.evaluateRandomly(encoder2,decoder2,inputlang,outputlang,pairs)

> in the meantime everybody just hold on .
=  与 此 同 时 ， 请 大 家 再 忍 耐 一 下 。   


/data/liwb/Documents/projects/machineTranslation/code/models.py:62: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  output, hidden = self.gru(output, hidden)



 0
[torch.cuda.LongTensor of size 1x1 (GPU 0)]



RuntimeError: invalid argument 6: wrong matrix size at /pytorch/torch/lib/THC/generic/THCTensorMathBlas.cu:453

# load valid data

In [2]:
import validDataProcess as vdp
h5py_file = h5py.File('../data/valid_data.h5py','r')
valid_pairs = h5py_file['pairs']
print(len(pairs))
print(pairs[0][0].decode('utf-8'))
print(pairs[0][1].decode('gb2312'))

valid_inputlang = vdp.Lang('en')
valid_outputlang = vdp.Lang('zh')
valid_inputlang.load('../data/en_valid1.pkl')
valid_outputlang.load('../data/zh_valid1.pkl')

####测试用，上面两行注释的语句在真正运行的时候要用到的##################################################################
# pairs = pairs[0:1000]
# for pair in pairs:
#     inputlang.addSentence(pair[0].decode('utf-8'))
#     outputlang.addSentence(pair[1].decode('gb2312'))

print(valid_inputlang.name,valid_inputlang.n_words)
print(valid_outputlang.name,valid_outputlang.n_words)
#h5py_file.close()

a pair of red crowned cranes have staked out their nesting territory
 一 对 丹 顶 鹤 正 监 视 着 它 们 的 筑 巢 领 地 
error: Name error------------------------------!
error: Name error------------------------------!
en 388091
zh 6754


In [3]:
print(pairs[90987][0].decode('utf-8'))

it ll be just like we planned .


In [6]:
pair = random.choice(pairs)
print(pair[0].decode('utf-8'))
print(pair[1].decode('gb2312'))

now if you ll just take a seat we can begin .
 好 ， 请 坐 吧 ， 这 样 我 们 就 可 以 开 始 了 。 


In [ ]:
import numpy as np
tt = ['我是中国人'.encode('gb2312'),'sdfeow dsf df dsf '.encode('utf-8')]
h5 = h5py.File('../data/test1.h5py','w')
h5.create_dataset('pairs',data=tt,dtype = 'S400')
h5.close()

#  data analysis

In [ ]:
count = inputlang.word2count
wordFreqLess10 = 0#保存词频小于某个阈值的词
wordFreqMore100 = 0
all_word_count=0
print('input english word count:' , inputlang.n_words)
for word in count:
    all_word_count += count[word]
    if count[word] <= 10:
        wordFreqLess10 +=count[word]
    if count[word] <= 10:
        wordFreqMore100 +=1
print('word count larger than 100:',inputlang.n_words - wordFreqMore100)
print('word count Less than 10:',wordFreqLess10)
print('rate of word count less than 10:', float(wordFreqLess10)/float(all_word_count))

## filter input language data

In [ ]:
testdict = {'a':23,'b':2,'c':4}
test2 = {1:'d',34:'f',22:'t'}
print(test2)
test2.pop(34)
print(test2)
a = testdict.copy()
print(a)
for word in testdict:
    if testdict[word] < 3:
        a.pop(word)
print(a)

### 匹配生僻字与非法字符

In [15]:
import re

def is_rare_name(string):
    pattern = re.compile(u"[~!@#$%^&* ]")
    match = pattern.search(string)
    if match:
        return True
    try:
        string.encode("gb2312")
    except UnicodeEncodeError:
        return True
    return False

def normalizeChinese(s):
    try:
        s.encode("gb2312")
    except UnicodeEncodeError:
        return ' '
    s = re.sub(r"[~!@#$%^&* ]+",r' ', s)
    return s


In [16]:
str1 = '我 是 @ 中 国 人 ， 我 # 不 才 囧 怪? 。'
#str1 = 'I an # a boy'
l = [normalizeChinese(s) for s in str1.split(' ')]
print(l)
tmp = ' '
for s in str1.split(' '):
    val = normalizeChinese(s)
    tmp += val+' '
print(tmp)

['我', '是', ' ', '中', '国', '人', '，', '我', ' ', '不', '才', ' ', '怪?', '。']
 我 是   中 国 人 ， 我   不 才   怪? 。 


In [ ]:
result = re.search(r'.','I love FishC.com!')
print(result.group())

In [ ]:
s = '我 是 @ 中 国 人 ， 我 # 不 囧 才 怪。'
s = re.sub(r"[~!@#$%^&* ]+",r' ', s)
print (s)

# build models

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

from hyperboard import Agent

agent = Agent(port=5100)
#agent = Agent(address='172.18.216.69',port=5000)
hyperparameters = {'test':0.1}
name = agent.register(hyperparameters, 'loss')

use_cuda = torch.cuda.is_available()
MAX_LENGTH = 30

# test filter data to model

In [ ]:
#返回词对应的下标
def indexesFromSentence(lang, sentence):
    #return [lang.word2index[word] for word in sentence.split(' ')]
    result = []
    all_lang_keys = lang.word2index.keys()
    for word in sentence.split(' '):
        if word in all_lang_keys:
            result.append(lang.word2index[word])
    return result

In [ ]:
test12 = 'a boy is watching tree.'
test11 = indexesFromSentence(inputlang, test12)
print(test11)


In [ ]:
testdict = {'a':1,'b':23,'c':4}
keys = testdict.keys()
list_result = []
for word in ['a','f','c','d']:
    if word in keys:
        list_result.append(testdict[word])
print(list_result)

# process valid data

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pandas as pd

import pickle
import jieba
import h5py

SOS_token = 0
EOS_token = 1



class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    #通过不断输入sentence（字符串的格式），构建词与下标的对应（词典），方便制作one-hot。
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def save(self, path):
        with open(path,'wb') as f:
            pickle.dump([self.name,self.word2index, self.word2count, self.index2word, self.n_words],f)
    
    def load(self,path):
        with open(path,'rb') as f:
            name, self.word2index, self.word2count, self.index2word, self.n_words = pickle.load(f)
        if self.name != name:
            print('error: Name error------------------------------!')
            
    def indexesFromSentence(self, sentence):
        indexes = []
        all_lang_keys = self.word2index.keys()
        for word in sentence.split(' '):
            if word in all_lang_keys:
                indexes.append(self.word2index[word])
        indexes.append(EOS_token)
        return indexes
            
##################################################################

def readFromFile(path):
    file = open(path)
    pattern = re.compile('<seg id=".*?"> (.*?) </seg>')
    result=[]
    for line in file.readlines():
        #print(line)
        item = re.findall(pattern,line)
        if item:
            #print(item[0])
            result.append(item[0])
    print('the number of all sentences is ', len(result))
    return result

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def normalizeChinese(s):
    try:
        s.encode("gb2312")
    except UnicodeEncodeError:
        return ' '
    s = re.sub(r"[~!@#$%^&* ]+",r' ', s)
    return s


#lang1 = 'zh'  lang2 = 'en'
#默认英文到中文
def readValidLangs(lang1, lang2, reverse=True, fenci=False):
    print("Reading lines...")
    zh_lines = readFromFile('../data/valid.en-zh.%s.sgm'% lang1)
    #zh_lines = zh_lines[0:20]  #for test
    
    zh_data_list = []
    if fenci:
        #jieba 分词
        for line in zh_lines:
            seg_line = jieba.cut(line,cut_all=False)
            #dic = [seg for seg in seg_line]
            dic = ' '.join(seg_line)
            tmp = ' '
            for char in dic.split(' '):
                val = normalizeChinese(char)
                tmp += val+' '
            zh_data_list.append(tmp)
    else: #用空格按字分开
        for line in zh_lines:
            dic = ' '.join(line)
            tmp = ' '
            for char in dic.split(' '):
                val = normalizeChinese(char)##去除生僻词
                tmp += val+' '
            zh_data_list.append(tmp)
            
    en_lines = readFromFile('../data/valid.en-zh.%s.sgm'% lang2)
    #en_lines = en_lines[0:20]  #for test
    
    # Split every line into pairs and normalize
    #去掉一些标点符号
    en_data_list = [[normalizeString(s) for s in l.split('\t')] for l in en_lines]
    pairs = []
    if reverse:
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
        for en,zh in zip(en_data_list,zh_data_list):
            input_lang.addSentence(en[0])
            output_lang.addSentence(zh)
            pairs.append([en[0].encode('utf-8'),zh.encode('gb2312')])
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        for en,zh in zip(en_data_list,zh_data_list):
            input_lang.addSentence(zh)
            output_lang.addSentence(en[0])
            pairs.append([zh.encode('gb2312'), en[0].encode('utf-8')])
            
    return input_lang, output_lang, pairs


In [9]:
#read data and process data
input_valid, output_valid, pairs = readValidLangs(lang1='zh', lang2='en')

#save data
input_valid.save('../data/en_valid1.pkl')
output_valid.save('../data/zh_valid1.pkl')

h5 = h5py.File('../data/valid_data.h5py','w')
h5.create_dataset('pairs',data=pairs,dtype = 'S400')
h5.close()

Reading lines...
the number of all sentences is  8000
the number of all sentences is  8000


In [6]:
import validDataProcess as vdp
h5py_file = h5py.File('../data/valid_data.h5py','r')
pairs = h5py_file['pairs']
print(len(pairs))
print(pairs[0][0].decode('utf-8'))
print(pairs[0][1].decode('gb2312'))

inputlang = vdp.Lang('en')
outputlang = vdp.Lang('zh')
inputlang.load('../data/en_valid1.pkl')
outputlang.load('../data/zh_valid1.pkl')

####测试用，上面两行注释的语句在真正运行的时候要用到的##################################################################
# pairs = pairs[0:1000]
# for pair in pairs:
#     inputlang.addSentence(pair[0].decode('utf-8'))
#     outputlang.addSentence(pair[1].decode('gb2312'))
    
print(inputlang.name,inputlang.n_words)
print(outputlang.name,outputlang.n_words)
#h5py_file.close()


8000
do you think we look young enough to blend in at a high school ?
 你 们 觉 得 我 们 看 起 来 够 年 轻 溜 进 高 中 吗 ？ 
en 8501
zh 2956


# process test data

In [12]:
test_lang, test_pairs = readTestLangs('../data/test.sgm')

Reading lines...
the number of all sentences is  8000


In [14]:
#save data
test_lang.save('../data/en_test.pkl')

h5 = h5py.File('../data/test_data.h5py','w')
h5.create_dataset('pairs',data=test_pairs,dtype = 'S400')
h5.close()

In [13]:
for pair in test_pairs:
    print(pair.decode('utf-8'))

b'if i had a nickel for every how about this ?'
b'someone who d have never forgiven me if i d have done that .'
b'anyway so lunch led to drinks which led to dinner '
b'and if you love my wife you should leave town .'
b'if they catch him they ll put him on trial for murder .'
b'gonna get paid twice by the feds when they give us the reward'
b'what was your reaction to protectors during the gulf war or vietnam ?'
b'you know contrary to what you might think about us . . .'
b'look i know you ve been chewed up by the system okay ?'
b'they found a gram of cocaine in the socialite s makeup campact .'
b'i came up to you i said hey dave i m attracted to men .'
b'if my father was here they would never shave my head .'
b'i can t claim to know how difficult this must be for you or how you feel .'
b'but you know you get older and you just don t want to hear the bad things .'
b'it s because you are a good time girl out to have fun .'
b'must be a big place . every prison gets paid by the prisoner .'
b

# calculate bleu score

In [10]:
import nltk
pair =  random.choice(pairs)
label_words = nltk.word_tokenize(pair[1].decode('gb2312'))
output_words = nltk.word_tokenize(pair[0].decode('utf-8'))
print(label_words)
print(output_words)
belu_score = nltk.translate.bleu_score.sentence_bleu([label_words],label_words)
print(belu_score)

['我', '也', '为', '你', '骄', '傲', '因', '为', '你', '尝', '试', '对', '这', '样', '的', '话', '题', '感', '兴', '趣', '。']
['and', 'i', 'm', 'proud', 'of', 'you', 'for', 'trying', 'to', 'be', 'interested', '.']
1.0


In [ ]:
import nltk
def evaluateValidData(encoder, decoder, inputlang, outputlang, pairs):
    predict_words = []
    bleu_score = 0.0
    for pair in pairs:
        output_words, attentions = evaluate(encoder, decoder, pair[0].decode('utf-8'),inputlang, outputlang)
        #output_sentence = ' '.join(output_words)
        label_words = nltk.word_tokenize(pair[1].decode('gb2312'))
        bleu_score += nltk.translate.bleu_score.sentence_bleu([label_words],output_words)
    bleu_score = bleu_score/len(pairs)
    return bleu_score
        

In [5]:

sos = 'EOS'.encode('gb2312')

In [6]:
print(sos.decode('gb2312'))

EOS
